In [134]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tweets/sample.csv


# **Data Loading And Exploration**

In [135]:
df = pd.read_csv('/kaggle/input/tweets/sample.csv')

df.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,@AppleSupport causing the reply to be disregar...,119236,NaN
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,@105835 Your business means a lot to us. Pleas...,NaN,119239.0
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,@76328 I really hope you all change but I'm su...,119238,NaN
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,@105836 LiveChat is online at the moment - htt...,119241,119242.0
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,@VirginTrains see attached error message. I've...,119243,119240.0


In [136]:
df.isna().sum()

tweet_id                    0
author_id                   0
inbound                     0
created_at                  0
text                        0
response_tweet_id          28
in_response_to_tweet_id    25
dtype: int64

In [137]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   tweet_id                 93 non-null     int64  
 1   author_id                93 non-null     object 
 2   inbound                  93 non-null     bool   
 3   created_at               93 non-null     object 
 4   text                     93 non-null     object 
 5   response_tweet_id        65 non-null     object 
 6   in_response_to_tweet_id  68 non-null     float64
dtypes: bool(1), float64(1), int64(1), object(4)
memory usage: 4.6+ KB


In [138]:
df.describe()

,tweet_id,in_response_to_tweet_id
count,93.000000,68.000000
mean,119285.451613,119285.676471
std,28.314045,28.910795
min,119237.000000,119239.000000
25%,119262.000000,119259.750000
50%,119285.000000,119284.500000
75%,119309.000000,119311.500000
max,119335.000000,119334.000000


# **Make texts lower case**

In [139]:
df['text_lower'] = df['text'].str.lower() 

df[['text', 'text_lower']].head()

,text,text_lower
0,@AppleSupport causing the reply to be disregar...,@applesupport causing the reply to be disregar...
1,@105835 Your business means a lot to us. Pleas...,@105835 your business means a lot to us. pleas...
2,@76328 I really hope you all change but I'm su...,@76328 i really hope you all change but i'm su...
3,@105836 LiveChat is online at the moment - htt...,@105836 livechat is online at the moment - htt...
4,@VirginTrains see attached error message. I've...,@virgintrains see attached error message. i've...


# **Punctuation Removal**

In [140]:
import string

print(type(string.punctuation))
print(string.punctuation)

<class 'str'>
!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [141]:
def remove_punctuation_from_text(text):
    text_translator = str.maketrans('','', string.punctuation)
    cleaned_text = text.translate(text_translator)
    return cleaned_text

# print(df.loc[0, "text"])

for index, data in df[['text_lower']].iterrows():
#     print(data['text_lower'])
    df.loc[index, 'text_no_punctuation'] = remove_punctuation_from_text(df.loc[index, 'text_lower'])
    
df[['text_no_punctuation']].head().iloc[1,0]

'105835 your business means a lot to us please dm your name zip code and additional details about your concern rr httpstcoznuu1vjn9r'

In [142]:
df[['text', 'text_lower', 'text_no_punctuation']].head()

,text,text_lower,text_no_punctuation
0,@AppleSupport causing the reply to be disregar...,@applesupport causing the reply to be disregar...,applesupport causing the reply to be disregard...
1,@105835 Your business means a lot to us. Pleas...,@105835 your business means a lot to us. pleas...,105835 your business means a lot to us please ...
2,@76328 I really hope you all change but I'm su...,@76328 i really hope you all change but i'm su...,76328 i really hope you all change but im sure...
3,@105836 LiveChat is online at the moment - htt...,@105836 livechat is online at the moment - htt...,105836 livechat is online at the moment https...
4,@VirginTrains see attached error message. I've...,@virgintrains see attached error message. i've...,virgintrains see attached error message ive tr...


# **Stopwords Removal**

In [143]:
import nltk # natural language toolkit
dir(nltk)

['AbstractLazySequence',
 'AffixTagger',
 'AlignedSent',
 'Alignment',
 'AnnotationTask',
 'ApplicationExpression',
 'Assignment',
 'BigramAssocMeasures',
 'BigramCollocationFinder',
 'BigramTagger',
 'BinaryMaxentFeatureEncoding',
 'BlanklineTokenizer',
 'BllipParser',
 'BottomUpChartParser',
 'BottomUpLeftCornerChartParser',
 'BottomUpProbabilisticChartParser',
 'Boxer',
 'BrillTagger',
 'BrillTaggerTrainer',
 'CFG',
 'CRFTagger',
 'CfgReadingCommand',
 'ChartParser',
 'ChunkParserI',
 'ChunkScore',
 'ClassifierBasedPOSTagger',
 'ClassifierBasedTagger',
 'ClassifierI',
 'ConcordanceIndex',
 'ConditionalExponentialClassifier',
 'ConditionalFreqDist',
 'ConditionalProbDist',
 'ConditionalProbDistI',
 'ConfusionMatrix',
 'ContextIndex',
 'ContextTagger',
 'ContingencyMeasures',
 'CoreNLPDependencyParser',
 'CoreNLPParser',
 'Counter',
 'CrossValidationProbDist',
 'DRS',
 'DecisionTreeClassifier',
 'DefaultTagger',
 'DependencyEvaluator',
 'DependencyGrammar',
 'DependencyGraph',
 'Depen

In [144]:
nltk.word_tokenize("ok i'm good")

['ok', 'i', "'m", 'good']

In [145]:
from nltk.corpus import stopwords
nltk.download('stopwords')

print(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any'

In [146]:
def remove_stopwords_from_text(text):
    words = nltk.word_tokenize(text)
    cleaned_text = " ".join([word for word in words if word not in stopwords.words('english')])
    return cleaned_text

for index, _ in df[['text_no_punctuation']].iterrows():
    df.loc[index, 'text_no_stopwords'] = remove_stopwords_from_text(df.loc[index, 'text_no_punctuation'])
    
print(df[['text_no_punctuation']].head().iloc[1,0])
print(df[['text_no_stopwords']].head().iloc[1,0])

105835 your business means a lot to us please dm your name zip code and additional details about your concern rr httpstcoznuu1vjn9r
105835 business means lot us please dm name zip code additional details concern rr httpstcoznuu1vjn9r


In [147]:
df[['text', 'text_lower', 'text_no_stopwords']].head()

,text,text_lower,text_no_stopwords
0,@AppleSupport causing the reply to be disregar...,@applesupport causing the reply to be disregar...,applesupport causing reply disregarded tapped ...
1,@105835 Your business means a lot to us. Pleas...,@105835 your business means a lot to us. pleas...,105835 business means lot us please dm name zi...
2,@76328 I really hope you all change but I'm su...,@76328 i really hope you all change but i'm su...,76328 really hope change im sure wont dont
3,@105836 LiveChat is online at the moment - htt...,@105836 livechat is online at the moment - htt...,105836 livechat online moment httpstcosy94vtu8...
4,@VirginTrains see attached error message. I've...,@virgintrains see attached error message. i've...,virgintrains see attached error message ive tr...


# **URLs Removal**

In [148]:
import re

In [149]:
def remove_url_from_text(text):
    url_rx_obj = re.compile(r'https?://\S+|www\.\S+') #.compile() allows you to compile a regular expression pattern into a regex object.
    clean_text = url_rx_obj.sub("", text) # Replace words matched by the pattern with ""
    return clean_text

for index, _ in df[['text_no_stopwords']].iterrows():
    df.loc[index, 'text_no_urls'] = remove_url_from_text(df.loc[index, 'text_no_stopwords'])
    
print(df[['text_no_stopwords']].head().iloc[3,0])
print(df[['text_no_urls']].head().iloc[3,0])

105836 livechat online moment httpstcosy94vtu8kq contact 03331 031 031 option 1 4 3 leave message request call back
105836 livechat online moment httpstcosy94vtu8kq contact 03331 031 031 option 1 4 3 leave message request call back


In [150]:
df[['text', 'text_lower', 'text_no_urls']].head()

,text,text_lower,text_no_urls
0,@AppleSupport causing the reply to be disregar...,@applesupport causing the reply to be disregar...,applesupport causing reply disregarded tapped ...
1,@105835 Your business means a lot to us. Pleas...,@105835 your business means a lot to us. pleas...,105835 business means lot us please dm name zi...
2,@76328 I really hope you all change but I'm su...,@76328 i really hope you all change but i'm su...,76328 really hope change im sure wont dont
3,@105836 LiveChat is online at the moment - htt...,@105836 livechat is online at the moment - htt...,105836 livechat online moment httpstcosy94vtu8...
4,@VirginTrains see attached error message. I've...,@virgintrains see attached error message. i've...,virgintrains see attached error message ive tr...


# **HTML Removal**

In [151]:
def remove_html_from_text(text):
    html_rx_obj = re.compile(r'<.*?>') #.compile() allows you to compile a regular expression pattern into a regex object.
    clean_text = html_rx_obj.sub("", text) # Replace words matched by the pattern with ""
    return clean_text

for index, _ in df[['text_no_urls']].iterrows():
    df.loc[index, 'text_no_html'] = remove_html_from_text(df.loc[index, 'text_no_urls'])
    
print(df[['text_no_urls']].iloc[5,0])
print(df[['text_no_html']].iloc[5,0])

105836 tried another device miriam mm
105836 tried another device miriam mm


# **Emoji Removals**

In [152]:
def remove_emoji_from_text(text):
    # Define a regular expression pattern to match emojis
    emoji_rx_obj = re.compile("["
        u"\U0001F600-\U0001F64F"  # Emoticons
        u"\U0001F300-\U0001F5FF"  # Symbols & Pictographs
        u"\U0001F680-\U0001F6FF"  # Transport & Map Symbols
        u"\U0001F700-\U0001F77F"  # Alchemical Symbols
        u"\U0001F780-\U0001F7FF"  # Geometric Shapes
        u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
        u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        u"\U0001FA00-\U0001FA6F"  # Extended-A
        u"\U0001FA70-\U0001FAFF"  # Extended-B
        u"\U0001F004-\U0001F0CF"  # Miscellaneous Symbols and Pictographs
        u"\U0001F170-\U0001F251"  # Enclosed Ideographic Supplement
        u"\U00002702-\U000027B0"  # Dingbats
                           "]+", flags=re.UNICODE)
    
    clean_text = emoji_rx_obj.sub("", text)
    return clean_text

for index, _ in df[['text_no_html']].iterrows():
    df.loc[index, 'text_no_emoji'] = remove_emoji_from_text(df.loc[index, 'text_no_html']) 
    
print(df[['text_no_html']].head().iloc[0,0])
print(df[['text_no_emoji']].head().iloc[0,0])

applesupport causing reply disregarded tapped notification keyboard opened😡😡😡
applesupport causing reply disregarded tapped notification keyboard opened


# **Stemming**

In [153]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
def stem_text(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

for index, _ in df[['text_no_emoji']].iterrows():
    df.loc[index, 'text_stemmed'] = stem_text(df.loc[index, 'text_no_emoji']) 
    
print(df[['text_no_html']].head().iloc[0,0])
print(df[['text_no_emoji']].head().iloc[0,0])
print(df[['text_stemmed']].head().iloc[0,0])

applesupport causing reply disregarded tapped notification keyboard opened😡😡😡
applesupport causing reply disregarded tapped notification keyboard opened
applesupport caus repli disregard tap notif keyboard open


# **Lemmatization**

In [ ]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

for index, _ in df[['text_no_emoji']].iterrows():
    df.loc[index, 'text_lemmatized'] = lemmatize_text(df.loc[index, 'text_no_emoji']) 
    
print(df[['text_no_html']].head().iloc[0,0])
print(df[['text_no_emoji']].head().iloc[0,0])
print(df[['text_lemmatized']].head().iloc[0,0])

In [164]:
import spacy

# Load the English NLP model from spaCy
nlp = spacy.load("en_core_web_sm")

def lemmatize_text(text):
    doc = nlp(text)
    return " ".join([word.lemma_ for word in doc])

for index, _ in df[['text_no_emoji']].iterrows():
    df.loc[index, 'text_lemmatized'] = lemmatize_text(df.loc[index, 'text_no_emoji']) 
    
print(df[['text_no_html']].head().iloc[0,0])
print(df[['text_no_emoji']].head().iloc[0,0])
print(df[['text_lemmatized']].head().iloc[0,0])

applesupport causing reply disregarded tapped notification keyboard opened😡😡😡
applesupport causing reply disregarded tapped notification keyboard opened
applesupport cause reply disregard tap notification keyboard open
